<a href="https://colab.research.google.com/github/vermilion4/GradedQuizes/blob/master/GradedQuizB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
#imports
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [103]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/energydata_complete.csv')
df.head()
df = df.drop(['date','lights'],axis=1)
df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [104]:
#relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6)
X = df['T2']
y = df['T6']

x_train,x_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
x_train= x_train.values.reshape(-1, 1)
y_train= y_train.values.reshape(-1, 1)
x_test = x_test.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

#model fit
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)

#prediction
predictions = linear_model.predict(x_test)

#r squared
from sklearn.metrics import r2_score
rsqu = r2_score(y_test,predictions)
round(rsqu,2)


0.64

In [105]:
#normalizin dataset with MinMaxScaler
scaler = MinMaxScaler()
normal_df = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
#input define
X = normal_df.drop(columns=['Appliances'])
#output define(target)
y = normal_df['Appliances']

#train_test split
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

#model fit
linear_model = LinearRegression().fit(x_train,y_train)

#prediction
prediction = linear_model.predict(x_test)


In [106]:
#mean absolute error
mae = mean_absolute_error(y_test,prediction)
mae.round(2)

0.05

In [107]:
#residual sum of squares
rss = np.sum((y_test - prediction)**2)
rss.round(2)

45.35

In [108]:
#mean squared error
mse = mean_squared_error(y_test,prediction)
mse.round(3)

0.008

In [109]:
#root mean squared error(rmse)
rmse = np.sqrt(mse)
rmse.round(3)

0.088

In [110]:
#r squared
rsqu = r2_score(y_test,prediction)
round(rsqu,2)

0.15

In [111]:
#ridge regression
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

predicti = ridge_reg.predict(x_test)
#ridge rmse
rmse = np.sqrt(mean_squared_error(y_test,predicti))
rmse.round(3)


0.088

In [112]:
#Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

pred = lasso_reg.predict(x_test)
#lasso rmse
rmse = np.sqrt(mean_squared_error(y_test,pred))
rmse.round(3)

0.094

In [113]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of the three features (lasso,ridge and linear model)
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

#function calls
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [114]:
#output weights
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.411071,-0.000000
1,T_out,-0.321860,-0.262172,0.000000
2,T2,-0.236178,-0.201397,0.000000
3,T9,-0.189941,-0.188916,-0.000000
4,RH_8,-0.157595,-0.156830,-0.000110
5,RH_out,-0.077671,-0.054724,-0.049557
6,RH_7,-0.044614,-0.045977,-0.000000
7,RH_9,-0.039800,-0.041367,-0.000000
8,T5,-0.015657,-0.019853,-0.000000
9,T1,-0.003281,-0.018406,0.000000


In [115]:
#feature with lowest weight
a = final_weights.groupby('Features')['Linear_Model_Weight'].sum()
a.nsmallest(1)

Features
RH_2   -0.456698
Name: Linear_Model_Weight, dtype: float64

In [116]:
#feature with highest weight
a = final_weights.groupby('Features')['Linear_Model_Weight'].sum()
a.nlargest(1)

Features
RH_1    0.553547
Name: Linear_Model_Weight, dtype: float64

In [117]:
#total non-zero lasso weights
final_weights[final_weights['Lasso_weight']!=0.0].count()

Features               4
Linear_Model_Weight    4
Ridge_Weight           4
Lasso_weight           4
dtype: int64